# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [ ]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=0"
#URL = "http://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=jobsite&salary=%2420%2C000%2B&radius=25&l=New+York&fromage=any&limit=50&sort=&psf=advsrch"

In [ ]:
import requests
import bs4
import urllib2
import numpy as np
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

import statsmodels.api as sm

In [ ]:
## YOUR CODE HERE

title=[]
company=[]
location=[]
salary=[]
summary=[]

def scraper(URL,city):
    dom = BeautifulSoup(requests.get(URL).text,"lxml")

    jobDivs = dom.find_all("div", {"class":"result","class":"row"})

    for div in jobDivs:
        divSoup = BeautifulSoup(unicode.join(u'\n',map(unicode,div)),"lxml")

        #get title
        title.append(divSoup.find("a",{"data-tn-element":"jobTitle"})['title'])

        #get company
        company1 = divSoup.find("a",{"data-tn-element":"companyName"})
        company2 = divSoup.find("span",{"class":"company"})
        company3 = divSoup.find("span",{"itemprop":"name"})
        if(company1 != None):
            #print "1: ",company1.contents[-1].strip()
            company.append(company1.contents[-1].strip())
        elif(company2 != None):
            #print "2: ",company2.strip()
            company.append(company2.text.strip())
        elif(company3 != None):
            if(company3.string != None):
                #print "3a: ",company3.string.strip()
                company.append(company3.string.strip())
            else:
                #print "3b: ",company3.contents[-1].string.strip()
                if(company3.contents[-1].string != None):
                    company.append(company3.contents[-1].string.strip())
                else:
                    company.append("What:" + company3.contents[-1].text)
                    
        else:
            print "N/A"

        #get location 
        #locationString = divSoup.find("span",{"class":"location"}).string
        #if (locationString == None):
        #    locationString = divSoup.find("span",{"itemprop":"addressLocality"}).contents[0].strip()
        location.append(city)

        #get salary

        salaryItem = divSoup.find("span",{"class":"location"}).findNextSibling("div")
        salaryItem2 = divSoup.find("td",{"class":"snip"}).findChild('nobr')
        if (salaryItem != None):
            salary.append(salaryItem.text)
        elif(salaryItem2 != None):
            #print "nobr: ", salaryItem2.string
            salary.append(salaryItem2.string)
        else:
            salary.append(np.nan)

        #get summary

        #print divSoup.find("span",{"class":"summary"}).text
        #summary.append(divSoup.find("span",{"class":"summary"}).text)



Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [ ]:
max_results_per_city = 500

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', "Atlanta","Seattle","Los+Angeles","Washington%2C+DC","Huntsville",
                 "San+Diego","Austin","Denver","St.+Louis","Boston","Portland","Philadelphia","Dallas","Houston","San+Antonio","Charlotte"]):
    print city
    for start in range(0, max_results_per_city, 10):
        print "Results: ", start
        url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l="+ city +"&start=" + str(start)
        scraper(url_template, city)

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [ ]:
print len(title)
print len(salary)
print len(location)
print len(salary)

In [ ]:
length = len(title)-1

In [ ]:
df = pd.DataFrame({'title':title[0:length], 'company':company[0:length], 'location':location[0:length], 'salary':salary[0:length]})

In [ ]:
df.describe()

In [ ]:
df = df.dropna()
df

Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [ ]:
yearly = pd.DataFrame(columns=('company', 'location', 'salary', 'title'))
for index, row in df.iterrows():
    if "a year" in row['salary']:
        yearly.loc[index] = row
#yearly

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [ ]:
import re

for index, row in yearly.iterrows():
    if "-" in row['salary']:
        tokenized = row['salary'].split("-")
        #print "First: ",re.sub("[^0-9]", "", tokenized[0].strip("$")).strip()
        #print "Second: ",re.sub("[^0-9]", "", tokenized[1].strip("$")).strip()
        lower = int(re.sub("[^0-9]", "", tokenized[0]).strip())
        higher = int(re.sub("[^0-9]", "", tokenized[1]).strip())
        average = (higher + lower)/2
        yearly.loc[index][2] = average
    else:
        yearly.loc[index][2] = int(re.sub("[^0-9]", "", row['salary']).strip())
        
    yearly.loc[index][1] = re.sub("\d+","", row['location']).strip() 
    
yearly.head()

In [47]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

yearly['salary'] = yearly['salary'].astype(int)
#print_full(yearly['salary'])

NameError: name 'yearly' is not defined

### Save your results as a CSV

In [182]:
yearly.describe()

yearly.to_csv("../assets/indeed_salary_data",encoding='utf-8',index=False)

NameError: name 'yearly' is not defined

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [48]:
import requests
import bs4
import urllib2
import numpy as np
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup

from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
import sklearn.metrics as metrics

import statsmodels.api as sm

In [49]:
data = pd.read_csv('../assets/indeed_salary_data')

In [50]:
data.head()

,company,location,salary,title
0,Next Step Systems,Chicago,100000,Junior Quantitative Research Analyst
1,Ginas Tech Jobs,Chicago,115000,Quantitative Research Analyst
2,Workbridge Associates,Chicago,105000,Mid-Level Data Scientist
3,Etech Hi Inc.,Chicago,71000,Statistician
4,Etech Hi Inc.,Chicago,85000,Sr. Statistician


In [51]:
data.count()

company     1014
location    1014
salary      1014
title       1014
dtype: int64

In [52]:
data = data.drop_duplicates()
data.count()

company     365
location    365
salary      365
title       365
dtype: int64

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [53]:
#import matplotlib.pyplot as plt
#%matplotlib inline

#data.plot.density()

In [54]:
meanSalary = np.mean(data["salary"])

def HigherOrLower(salary):
    if salary > meanSalary:
        return 1
    else:
        return 0

data["HighLow"] = map(HigherOrLower,data["salary"])
smdata = data
smdata["Intercept"] = 1
#data["Intercept"] = 1

In [55]:
def categorizeTitles(string):
    title = string.lower()
    if("data scientist" in title):
        return 0
    elif("statistician" in title):
        return 1
    elif("data analyst" in title):
        return 2
    elif("data engineer" in title):
        return 3
    elif("research analyst" in title):
        return 4
    elif("software" in title or "engineer" in title or "developer" in title):
        return 5
    elif("analyst" in title):
        return 6
    elif("director" in title):
        return 7
    elif("manager" in title):
        return 8
    elif("scientist" in title):
        return 9
    else:
        return 10
    
categorizedTitles = map(categorizeTitles,data.title)
data["categorizedTitle"] = categorizedTitles
smdata["categorizedTitle"] = categorizedTitles

#### Thought experiment: What is the baseline accuracy for this model?

In [56]:
locationData = data.groupby("location",as_index="False").count()
mostCommonLocation = locationData.idxmax()[1]
print "Most Common Location: ", mostCommonLocation

companyData = data.groupby("company",as_index="False").count()
mostCommonCompany = companyData.idxmax()[1]
print "Most Common Company: ", mostCommonCompany

titleData = data.groupby("categorizedTitle",as_index="False").count()
mostCommonTitle = titleData.idxmax()[1]
print "Most Common Categorized Title: ", mostCommonTitle

#titleData

Most Common Location:  Washington%C+DC
Most Common Company:  Workbridge Associates
Most Common Categorized Title:  0


#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [75]:
categories = ["location","company","categorizedTitle"]
for category in categories:
    series = data[category]
    dummies = pd.get_dummies(series, prefix=category)
    data = pd.concat([data, dummies], axis=1)
    if(categories != "company"):
        smdata = pd.concat([smdata, dummies], axis = 1)
    
#delete most common location for colinearity
del data["location_"+mostCommonLocation]
del data["company_"+mostCommonCompany]
del data["categorizedTitle_"+str(mostCommonTitle)]

del smdata["location_"+mostCommonLocation]
del smdata["company_"+mostCommonCompany]
del smdata["categorizedTitle_"+str(mostCommonTitle)]


In [76]:
#data

In [77]:
X = data.iloc[:,7:]
y = data["HighLow"]
y.name = "Actual"

### Normal


In [78]:
logiReg = LogisticRegression()
model = logiReg.fit(X,y)
predictions = model.predict(X)
print "Model score: ", model.score(X,y)

Model score:  0.931506849315


In [79]:
print "Accuracy score: ", metrics.accuracy_score(y,predictions)
print "Precision score: ", metrics.precision_score(y,predictions)
print "Recall score: ", metrics.recall_score(y,predictions)

Accuracy score:  0.931506849315
Precision score:  0.914285714286
Recall score:  0.941176470588


In [80]:
pd.crosstab(y,predictions)


col_0,0,1
Actual,,
0,180,15
1,10,160


#### Lasso

In [81]:
logiReg = LogisticRegression()
logiReg.penalty = "l1"
model = logiReg.fit(X,y)
predictions = model.predict(X)
print "Model score: ", model.score(X,y)

Model score:  0.761643835616


In [82]:
print "Accuracy score: ", accuracy_score(y,predictions)
print "Precision score: ", precision_score(y,predictions)
print "Recall score: ", recall_score(y,predictions)

Accuracy score:  0.761643835616
Precision score:  0.761006289308
Recall score:  0.711764705882


In [83]:
pd.crosstab(y,predictions)

col_0,0,1
Actual,,
0,157,38
1,49,121


### Ridge

In [84]:
logiReg = LogisticRegression()
logiReg.penalty = "l2"
model = logiReg.fit(X,y)
predictions = model.predict(X)
print "Model score: ", model.score(X,y)

Model score:  0.931506849315


In [85]:
print "Accuracy score: ", accuracy_score(y,predictions)
print "Precision score: ", precision_score(y,predictions)
print "Recall score: ", recall_score(y,predictions)

Accuracy score:  0.931506849315
Precision score:  0.914285714286
Recall score:  0.941176470588


In [86]:
pd.crosstab(y,predictions)

col_0,0,1
Actual,,
0,180,15
1,10,160


In [90]:
smX = smdata.iloc[:,7:]
smy = smdata["HighLow"]
smy.name = "Actual"


logit = sm.Logit(smy, smX)
smdata.columns.values
result = logit.fit_regularized()
result.summary()

KeyboardInterrupt: 

In [ ]:
%matplotlib

sns.lmplot("location_New+York","HighLow",data, logistic=True)

In [ ]:
np.exp(result.params).sort_values()
conf = result.conf_int()
conf["OR"]  = result.params
conf.columns = ['2.5%','97.5','OR']
np.exp(conf).sort_values('OR')

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [ ]:
## YOUR CODE HERE

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [ ]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [ ]:
## YOUR CODE HERE

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [ ]:
## YOUR CODE HERE

In [ ]:
## YOUR CODE HERE

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [ ]:
## YOUR CODE HERE

In [ ]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [ ]:
## YOUR CODE HERE